In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
dbutils.widgets.text("target_dataset", "VCM_DWH_PRD") #udp_wcm_gold_vcm_dwh
dbutils.widgets.text("target_table", "f_movement_po")
dbutils.widgets.text("metadata_schema", "udp_wcm_metadata_dev")
dbutils.widgets.text(
    "dependency_table","VCM_STG_INF.SAP_VCDMMIM03"
)

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
dbutils.widgets.text("field_calday", "CALDAY")
dbutils.widgets.text("field_id", "HASH_ID")

In [0]:
field_calday = dbutils.widgets.get("field_calday")
field_id = dbutils.widgets.get("field_id")

print(f"field_calday: {field_calday}")
print(f"field_id: {field_id}")

In [0]:
environment = dbutils.widgets.get("environment")
target_dataset = dbutils.widgets.get("target_dataset")
target_table = dbutils.widgets.get("target_table")
metadata_schema = dbutils.widgets.get("metadata_schema")
dependency_table = dbutils.widgets.get("dependency_table")
dependency_table = [x.strip().upper() for x in dependency_table.split(",")]
dependency_table = "'" + "','".join(dependency_table) + "'"

print(f"environment: {environment}")
print(f"target_dataset: {target_dataset}")
print(f"target_table: {target_table}")
print(f"catalog_name: {catalog_name}")
print(f"metadata_schema: {metadata_schema}")
print(f"dependency_table: {dependency_table}")

In [0]:
%run "../common/common_etl_load" 

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.f_movement_po
(
  hash_id BIGINT,
  calday DATE,
  po_doc STRING,
  po_item BIGINT,
  gr_date DATE,
  gr_qty FLOAT,
  gr_value FLOAT
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
if CALDAY_IN_STR != "":
    spark.sql(f"""
    CREATE OR REPLACE TEMP VIEW temp_{target_table}
    AS
    WITH 
    main AS (
        SELECT 
            calday, 
            oi_ebeln,
            oi_ebelp,
            movetype,
            zc_ltstgr,
            dcindic,
            cpquabu,
            cppvlc,
            segment,
            plant,
            move_plant
        FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vcdmmim03
        WHERE CALDAY IN ({CALDAY_IN_STR}) and stocktype = 'A' 
            AND movetype IN ('101','102', '122', '123')
            AND (
                segment IN ('1100', '1500', '2000', '1009')
                OR plant IN (
                    SELECT store_id 
                    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store 
                    WHERE business_unit = 'FCDC'
                )
                OR move_plant IN (
                    SELECT store_id 
                    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store 
                    WHERE business_unit = 'FCDC'
                )
            )
    )
    SELECT 
        farm_fingerprint(CONCAT(
            IFNULL(CAST(calday AS STRING), ""), 
            IFNULL(CAST(oi_ebeln AS STRING), ""), 
            IFNULL(CAST(oi_ebelp AS STRING), "")
        )) AS hash_id,
        calday,
        oi_ebeln AS po_doc,
        oi_ebelp AS po_item,
        MAX(IF(movetype = '101', zc_ltstgr, NULL)) AS gr_date,
        SUM(CASE WHEN dcindic = 'H' THEN -1 ELSE 1 END * ABS(cpquabu)) AS gr_qty,
        SUM(CASE WHEN dcindic = 'H' THEN -1 ELSE 1 END * ABS(cppvlc)) AS gr_value
    FROM main 
    GROUP BY calday, oi_ebeln, oi_ebelp;
    """)
else:
    print(f"No valid calday's found In common-etl to create the view: temp_{target_table}")

In [0]:
if CALDAY_IN_STR != '':
    calday_query = f"""
        SELECT 
            CONCAT("'", CONCAT_WS("','", COLLECT_SET(CAST(TO_DATE(calday, 'yyyy-MM-dd') AS STRING))), "'") AS calday_in_str
        FROM temp_{target_table}
        """
    calday_df = spark.sql(calday_query)
    calday_in_str = calday_df.collect()[0][0]
else:
    print(f"No valid calday's found In common-etl to create the view: temp_{target_table}")

In [0]:
if CALDAY_IN_STR != "":
    # Get the calday list string from temp table
    calday_query = f"""
    SELECT
        CONCAT("'", CONCAT_WS("','", COLLECT_SET(CAST(TO_DATE(calday, 'yyyy-MM-dd') AS STRING))), "'") AS calday_in_str
    FROM temp_{target_table}
    """

    calday_df = spark.sql(calday_query)
    calday_in_str = calday_df.collect()[0][0]
    print(calday_in_str)

    # Only run DELETE if the string is non-empty
    if calday_in_str.strip("'"):  # checks if there's any date inside the quotes
        spark.sql(
            f"""
        DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_movement_po
        WHERE calday IN ({calday_in_str})
        """
        ).display()
    else:
        print("no valid calday's found to delete.")
else:
    print(
        f"temp_{target_table} could't be created as no valid calday's found In common-etl so {catalog_name}.udp_wcm_gold_vcm_dwh.f_movement_po can't be deleted using calday's from temp_{target_table}"
    )

In [0]:
if CALDAY_IN_STR != "":
    # Insert rows from temp table into main table
    INSERT_ROW = ", ".join(
        spark.table(f"{catalog_name}.udp_wcm_gold_vcm_dwh.f_movement_po").columns
    )
    spark.sql(
        f"""
        INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.f_movement_po ({INSERT_ROW})
        SELECT {INSERT_ROW} FROM temp_{target_table}
    """
    ).display()
else:
    print(
        f"temp_{target_table} could't be created as no valid calday's found In common-etl so {catalog_name}.udp_wcm_gold_vcm_dwh.f_movement_po can't be inserted using calday's from temp_{target_table}"
    )

In [0]:
spark.sql(f"""DROP VIEW IF EXISTS temp_{target_table}""")

In [0]:
if CALDAY_IN_STR == '':
    dbutils.notebook.exit("No valid calday's found In common-etl to insert data into Table: ETL_LOG_DEPENDENCIES and ETL_DELTA_TABLE and ")

In [0]:
%run "../common/common_etl_update"